In [10]:

import os
import requests
from typing import List
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [1]:
import pandas as pd
merged_data = pd.read_parquet('/Users/soumyasouravbehera/Downloads/final_data_dropped.parquet', engine = 'pyarrow')

In [3]:
merged_data.columns

Index(['PATIENT', 'DESCRIPTION', 'REASONDESCRIPTION', 'BIRTHDATE', 'DEATHDATE',
       'DESCRIPTION_allergy', 'START_careplan', 'STOP_careplan',
       'DESCRIPTION_careplan', 'REASONDESCRIPTION_careplan', 'START_condition',
       'STOP_condition', 'DESCRIPTION_condition', 'DATE_immunization',
       'DESCRIPTION_immunization', 'DESCRIPTION_procedures'],
      dtype='object')

In [16]:
# Initialize and constants

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [17]:
import pandas as pd
import openai

# Set your OpenAI API key
openai.api_key = api_key
# Function to create the prompt
def create_prompt(patient_data):
    return f"""
    Generate a detailed markdown summary for the following patient details:
    - Patient ID: {patient_data['PATIENT']}
    - Description: {patient_data['DESCRIPTION']}
    - Birthdate: {patient_data['BIRTHDATE']}
    - Deathdate: {patient_data['DEATHDATE']}
    - Allergies: {patient_data['DESCRIPTION_allergy'] or "None"}
    - Careplans: {patient_data['DESCRIPTION_careplan'] or "None"}
    - Conditions: {patient_data['DESCRIPTION_condition'] or "None"}
    - Immunizations: {patient_data['DESCRIPTION_immunization'] or "None"}
    - Procedures: {patient_data['DESCRIPTION_procedures'] or "None"}
    
    Ensure the output includes:
    1. Clear sections for each category.
    2. Insights on the patient's data, e.g., if any critical details are missing or require attention.
    3. Suggestions for next steps based on their conditions, if applicable.
    """

# Function to get markdown from OpenAI
def generate_markdown_summary(patient_id):
    # Filter the dataset for the patient
    patient_data = merged_data[merged_data['PATIENT'] == patient_id]
    
    if patient_data.empty:
        return f"No data found for patient ID {patient_id}"
    
    # Convert to dictionary for easier access
    patient_data = patient_data.iloc[0].to_dict()
    
    # Create the prompt
    prompt = create_prompt(patient_data)
    
    # Generate the markdown summary using OpenAI
    try:
        response = openai.chat.completions.create(
            model="gpt-4o-mini",  # Use a suitable model
            messages=[
                {"role": "system", "content": "You are a helpful assistant skilled in summarizing patient details."},
                {"role": "user", "content": prompt}
            ]
        )
        result =  response.choices[0].message.content
        return display(Markdown(result))
    except Exception as e:
        return f"Error generating markdown: {str(e)}"



# Input: Specify the patient ID
patient_id = "bb66df6a-fe47-4be9-804f-e896ab535b99"
markdown_details = generate_markdown_summary(patient_id)


# # Optionally, save the output to a file
# with open(f'patient_{patient_id}_summary.md', 'w') as file:
#     file.write(markdown_details)


# Patient Summary

## Patient Information
- **Patient ID:** bb66df6a-fe47-4be9-804f-e896ab535b99
- **Birthdate:** November 16, 1948
- **Deathdate:** Not Applicable (alive)

## Encounter Details
- **Type of Encounter:** Outpatient Encounter

## Allergies
- **Allergies:** None reported

## Care Plans
- **Care Plans:** None available

## Medical Conditions
- **Conditions:** None documented

## Immunizations
- **Immunizations Received:**
  - **Influenza:** Seasonal, injectable, preservative-free

## Procedures
- **Procedures:** None performed

---

## Insights
The patient has no reported allergies, chronic conditions, care plans, or recent procedures. They have received a seasonal influenza vaccination, which is commendable, especially considering their age. 

**Missing Information:**
- There are no current medical conditions documented, which could suggest either good health or a lack of thorough examination/documentation.
- No care plans are in place, indicating a potential need for routine health monitoring or preventive care.

## Next Steps
1. **Regular Health Monitoring:** Given the patient's age, it would be prudent to schedule routine check-ups to monitor blood pressure, cholesterol levels, and other key health indicators.
  
2. **Discuss Preventive Screenings:** Consider discussing age-appropriate screenings such as mammograms, colonoscopies, or prostate checks depending on gender, family history, and other risk factors.
  
3. **Vaccination Update:** Review potential need for other vaccinations, such as pneumococcal vaccines or shingles vaccination, depending on the patient’s health status and medical history.

4. **Health Education:** Provide education on maintaining a healthy lifestyle, including diet, exercise recommendations, and the importance of regular medical follow-ups.

5. **Documentation Improvement:** Ensure that all relevant health information, care plans, and conditions are documented in the medical record during the next visit for better health management.

Generated Markdown Summary:
None
